<a href="https://colab.research.google.com/github/likhitha-dhanaraju/smart-assist-chatbot/blob/model_development/01_QnA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing and Improrting Libraries

In [2]:
import os
import pickle
import json
import re
import string
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Optionally, you can reset the warning filters later if needed
# warnings.resetwarnings()

## Data Preprocessing

In [3]:
root = '/content/drive/MyDrive/03 EDHEC/Master Thesis/QnA model'

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
from transformers import T5Tokenizer
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, contexts, questions, answers, tokenizer, max_length=1024):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        question = self.questions[idx]
        answer = self.answers[idx]

        input_text = f"question: {question} context: {question}"
        target_text = answer

        # Tokenize input and target text
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=self.max_length, pad_to_max_length=True, truncation=True)
        target_ids = self.tokenizer.encode(target_text, return_tensors="pt", max_length=self.max_length, pad_to_max_length=True, truncation=True)

        # Ensure both input and target sequences have the same length
        max_seq_length = max(input_ids.size(1), target_ids.size(1))

        input_ids = input_ids[:, :max_seq_length]
        target_ids = target_ids[:, :max_seq_length]

        return {"input_ids": input_ids.squeeze(0), "target_ids": target_ids.squeeze(0)}


## With preparing dataset

In [6]:
contexts_text = open(os.path.join(root, 'context_file.txt'), 'r', encoding='utf-8').read()
answers_text = open(os.path.join(root, 'answers_file.txt'), 'r', encoding='utf-8').read()
questions_text = open(os.path.join(root, 'questions_file.txt'), 'r', encoding='utf-8').read()

contexts = contexts_text.split('\n')
answers = answers_text.split('\n')
questions = questions_text.split('\n')

In [7]:
# Prepare custom dataset
custom_dataset = CustomDataset(contexts, questions, answers, tokenizer)
custom_dataset

In [28]:
from torch.utils.data import random_split

train_size = int(0.01 * len(custom_dataset))
test_size = len(custom_dataset) - train_size

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

In [31]:
len(train_dataset)

42660

In [30]:
import pickle

# Save custom dataset
with open(os.path.join(root, "QnA_train_dataset_1_percent.pkl"), "wb") as f:
    pickle.dump(train_dataset, f)

# # Save custom dataset
# with open(os.path.join(root, "QnA_test_dataset.pkl"), "wb") as g:
#     pickle.dump(test_dataset, g)

# # Load custom dataset
# with open("custom_dataset.pkl", "rb") as f:
#     loaded_custom_dataset = pickle.load(f)

## Without preparing dataset

In [35]:
# Load custom dataset
with open(os.path.join(root, "QnA_train_dataset_1_percent.pkl"), "rb") as f:
    train_dataset = pickle.load(f)

In [36]:
from torch.utils.data import DataLoader

# Create a DataLoader for the dataset
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

## Model training

In [37]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [ ]:
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

model.to(device)

# Training loop
model.train()
num_epochs = 20
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for batch in tqdm(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, labels=target_ids)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Loss: {loss.item()}")


Epoch 1/20


 85%|████████▍ | 906/1067 [12:41<02:15,  1.19it/s]

In [ ]:
# Save the trained model
model.save_pretrained(os.path.join(root, "trained_t5_model_1_percent_data_epoch 10"))

## Evaluating the model

In [ ]:
# Example input text
context = "Product title is Perler Beads Assorted Multicolor Fuse Beads for Kids Crafts, 11000 pcs. Product overview is Color is Multicolor Material is Plastic Size is 11,000 Count Brand is Perler Shape is Round Item Weight is 1.69 Pounds Number of Pieces is 11000. Product description is Includes (11000) assorted Perler fuse beads and reusable ironing paper in plastic storage jar. This mega set of 11,000 Perler fuse beads comes with 30 different colors, including toothpaste, pastel lavender, butterscotch, and neon pink. Use your assorted Perler fuse beads with pre-made Perler bead design or get creative and make your own. These multicolor Perler beads are great arts and crafts activity for children. Use Perler pegboards, ironing paper, and an iron to complete your craft. Multicolor Perler beads set suitable for ages and up."
question = "What is the Color of the product?"

# Concatenate context and question using a separator token
input_text = f"context: {context} question: {question}"

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate answer
with torch.no_grad():
    output = model.generate(input_ids, max_length=1024)  # Adjust max_length as needed
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("Input Text:", input_text)
print("Generated Answer:", answer)

Checking answer w.r.t pre-trained model

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
original_tokenizer = T5Tokenizer.from_pretrained(model_name)
original_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Example input text
context = "Product title is Perler Beads Assorted Multicolor Fuse Beads for Kids Crafts, 11000 pcs. Product overview is Color is Multicolor Material is Plastic Size is 11,000 Count Brand is Perler Shape is Round Item Weight is 1.69 Pounds Number of Pieces is 11000. Product description is Includes (11000) assorted Perler fuse beads and reusable ironing paper in plastic storage jar. This mega set of 11,000 Perler fuse beads comes with 30 different colors, including toothpaste, pastel lavender, butterscotch, and neon pink. Use your assorted Perler fuse beads with pre-made Perler bead design or get creative and make your own. These multicolor Perler beads are great arts and crafts activity for children. Use Perler pegboards, ironing paper, and an iron to complete your craft. Multicolor Perler beads set suitable for ages and up."
question = "What is the Color of the product?"

# Concatenate context and question using a separator token
input_text = f"context: {context} question: {question}"

# Tokenize input text
input_ids = original_tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate answer
with torch.no_grad():
    output = original_model.generate(input_ids, max_length=1024)  # Adjust max_length as needed
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("Input Text:", input_text)
print("Generated Answer:", answer)